# MACHINE LEARNING
## AUTHOR: ANTE DUJIC
<hr style="border:2px solid gray"> </hr>

<center><img src="https://learn.microsoft.com/en-us/windows/ai/images/winml-graphic.png" width="800"/></center>

<hr style="border:2px solid gray"> </hr>